In [9]:
"""
1. 定义模型添加dropout层
"""
from mxnet import gluon
from mxnet import autograd

drop_prob1 = .2
drop_prob2 = .5

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(256, activation= 'relu'))
    if autograd.is_training:
        net.add(gluon.nn.Dropout(drop_prob1))
    net.add(gluon.nn.Dense(256, activation= 'relu'))
    if autograd.is_training:
        net.add(gluon.nn.Dropout(drop_prob2))
    net.add(gluon.nn.Dense(10))
net.initialize()    

In [10]:
"""
2. 训练
"""
import sys
sys.path.append('..')
import utils
from mxnet import ndarray as nd

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.5})

for epoch in range(200):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += utils.accuracy(output, label)

    test_acc = utils.evaluate_accuracy(test_data, net)
    if epoch% 5 ==0:
        print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (epoch, train_loss/len(train_data),
                                                                 train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.817604, Train acc 0.693710, Test acc 0.811498
Epoch 5. Loss: 0.376432, Train acc 0.862814, Test acc 0.869391
Epoch 10. Loss: 0.322631, Train acc 0.880459, Test acc 0.878405
Epoch 15. Loss: 0.289582, Train acc 0.892261, Test acc 0.887220
Epoch 20. Loss: 0.271864, Train acc 0.899172, Test acc 0.894832
Epoch 25. Loss: 0.250761, Train acc 0.905599, Test acc 0.894932
Epoch 30. Loss: 0.238235, Train acc 0.909572, Test acc 0.893129
Epoch 35. Loss: 0.224854, Train acc 0.915698, Test acc 0.897336
Epoch 40. Loss: 0.216804, Train acc 0.916800, Test acc 0.895232
Epoch 45. Loss: 0.208390, Train acc 0.921725, Test acc 0.898137
Epoch 50. Loss: 0.200795, Train acc 0.924813, Test acc 0.898738
Epoch 55. Loss: 0.191592, Train acc 0.926816, Test acc 0.903546
Epoch 60. Loss: 0.183124, Train acc 0.929671, Test acc 0.899439
Epoch 65. Loss: 0.180312, Train acc 0.932325, Test acc 0.901042
Epoch 70. Loss: 0.176590, Train acc 0.932192, Test acc 0.901142
Epoch 75. Loss: 0.166686, Train acc 0.9362